<a href="https://colab.research.google.com/github/Mombinjenga/Potato-Disease-Detector/blob/main/Potato_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving potato-dataset.zip to potato-dataset.zip


In [ ]:
import zipfile

with zipfile.ZipFile("potato-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


In [ ]:
import os
print(os.listdir("/content/dataset"))
dataset_path = "/content/dataset/PlantVillage"
print(os.listdir(dataset_path))



['PlantVillage']
['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']


In [ ]:
# Loading Dataset
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,    # 80% train, 20% val
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)


Found 2152 files belonging to 3 classes.
Using 1722 files for training.
Found 2152 files belonging to 3 classes.
Using 430 files for validation.


In [ ]:
# Checking class counts
import numpy as np

def count_images(dataset):
    counts = {class_name:0 for class_name in dataset.class_names}
    for images, labels in dataset.unbatch():
        counts[dataset.class_names[labels.numpy()]] += 1
    return counts

print("Training set counts:", count_images(train_ds))
print("Validation set counts:", count_images(val_ds))


Training set counts: {'Potato___Early_blight': 796, 'Potato___Late_blight': 803, 'Potato___healthy': 123}
Validation set counts: {'Potato___Early_blight': 204, 'Potato___Late_blight': 197, 'Potato___healthy': 29}


In [ ]:
# Prefetch perfomance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
